In [29]:
# Import live code changes in
%load_ext autoreload
%autoreload 

from pathlib import Path
import os
import xarray as xr
# Disable warnings for data download via API
import urllib3 
urllib3.disable_warnings()
# Disable xarray runtime warnings
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
import pandas as pd
import numpy as np
from scipy.interpolate import LinearNDInterpolator
from lmoments3 import distr
from scipy.stats import gumbel_r, kstest
from tqdm import tqdm
import math
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set filepaths and provide data info

In [30]:
root = Path.cwd().parent # find project root
risk_basin_path = os.path.join(root, 'outputs', 'flood', 'risk', 'basins', 'risk_basins.csv')
basin_outlets = os.path.join(root, 'inputs', 'flood', 'dependence', 'lev06_outlets_final_clipped_Thailand_constrained.csv')
risk_data = pd.read_csv(risk_basin_path)
# Drop first "unnamed column"
risk_data = risk_data.iloc[:, 1:]
# Add AEP column
risk_data['AEP'] = 1 / risk_data['RP']
# Add a column converting current prorection level into AEP
risk_data['Pr_L_AEP'] = np.where(risk_data['Pr_L'] == 0, 0, 1 / risk_data['Pr_L']) # using numpy where avoids zero division errors
risk_data.reset_index(drop=True, inplace=True)

In [31]:
risk_data.head()

,FID,GID_1,NAME,HB_L6,Pr_L,damages,RP,AEP,Pr_L_AEP
0,0,THA.62_1,Si Sa Ket,4.061140e+09,16.387501,0.0,10,0.1,0.061022
1,1,THA.72_1,Ubon Ratchathani,4.061140e+09,20.812099,0.0,10,0.1,0.048049
2,2,THA.5_1,Buri Ram,4.061131e+09,16.739300,0.0,10,0.1,0.059740
3,3,THA.71_1,Trat,4.061131e+09,18.360399,0.0,10,0.1,0.054465
4,4,THA.5_1,Buri Ram,4.061131e+09,16.739300,0.0,10,0.1,0.059740


In [53]:
basins = pd.read_csv(basin_outlets)
basin_ids = basins['HYBAS_ID_L6'].to_list()

In [54]:
@dataclass
class BasinComponent:
    admin_id: str              # GID_1 or similar
    aeps: np.ndarray            # annual exceedance probabilities
    baseline_losses: np.ndarray # baseline flood losses
    # adapted_losses: np.ndarray # adapted flood losses (urban areas masked)
    protection_aep: float       # New_Pr_L or Pr_L
    meta: dict = None

    def baseline_loss_at(self, aep_event: float) -> float:
        return float(np.interp(aep_event, self.aeps, self.baseline_losses))

    def baseline_loss_at(self, aep_event: float) -> float:
        return float(np.interp(aep_event, self.aeps, self.baseline_losses))

    def protected_loss(self, aep_event: float) -> float:
        """Apply protection threshold inside MC simulation."""
        if aep_event > self.protection_aep:  # protected
            return 0.0
        else:
            return self.baseline_loss_at(aep_event)

    def adapted_loss(self, aep_event:float) -> float:
        if aep_event > self.protectioni_aep:
            return 0.0
        else:
            return self.adapted_loss_at(aep_event)


In [55]:
@dataclass
class BasinLossCurve:
    basin_id: int
    components: list[BasinComponent]

    def loss_at_event_aep(self, aep_event: float) -> float:
        return sum(c.protected_loss(aep_event) for c in self.components)

In [56]:
def build_basin_curves(df):
    basin_dict = {}

    # group by basin AND component (admin-level)
    for (basin_id, admin_id), g in df.groupby(["HB_L6", "GID_1"]):

        aeps = g["AEP"].to_numpy()
        losses = g["damages"].to_numpy()

        # Add 2-year (bankfull) flood equivalent to 0 risk
        aeps = np.concatenate(([0.5], aeps))
        losses = np.concatenate(([0.0], losses))
        # sort so curve is monotonic in RP
        order = np.argsort(-aeps)
        aeps = aeps[order]
        losses = losses[order]

        # choose your protection field
        prot = g["Pr_L_AEP"].iloc[0]

        component = BasinComponent(
            admin_id=admin_id,
            aeps=aeps,
            baseline_losses=losses,
            protection_aep=prot,
        )

        if basin_id not in basin_dict:
            basin_dict[basin_id] = []
        basin_dict[basin_id].append(component)

    # wrap each basin into a BasinLossCurve
    return {
        basin_id: BasinLossCurve(basin_id=basin_id, components=components)
        for basin_id, components in basin_dict.items()
    }

In [59]:
basin_curves: dict[int, BasinLossCurve] = build_basin_curves(risk_data)

In [73]:
# Run simple simulation
n_years = 1
seed = 0
rng = np.random.default_rng(seed)
basin_ids = list(basin_curves.keys())
national_losses = np.zeros(n_years, dtype=float)
for t in range(n_years):
    total_loss = 0.0
    for basin_id in basin_ids:
        curve = basin_curves[basin_id]
        # Sample a probability
        aep_event = rng.uniform(0.0, 1.0)
        basin_loss = curve.loss_at_event_aep(aep_event)
        print('ID:', basin_id, 'P:', aep_event, 'Loss:', basin_loss)
        total_loss += basin_loss
    national_losses[t] = total_loss

ID: 4060017870.0 P: 0.6369616873214543 Loss: 0.0
ID: 4060018070.0 P: 0.2697867137638703 Loss: 0.0
ID: 4060018080.0 P: 0.04097352393619469 Loss: 571246307.0
ID: 4060018230.0 P: 0.016527635528529094 Loss: 15536838408.003906
ID: 4060018240.0 P: 0.8132702392002724 Loss: 0.0
ID: 4060018280.0 P: 0.9127555772777217 Loss: 0.0
ID: 4060018290.0 P: 0.6066357757671799 Loss: 0.0
ID: 4060018330.0 P: 0.7294965609839984 Loss: 0.0
ID: 4060018340.0 P: 0.5436249914654229 Loss: 0.0
ID: 4060018350.0 P: 0.9350724237877682 Loss: 0.0
ID: 4060018360.0 P: 0.8158535541215322 Loss: 0.0
ID: 4060018370.0 P: 0.002738500170148095 Loss: 75534504.0
ID: 4060018410.0 P: 0.8574042765875693 Loss: 0.0
ID: 4060018420.0 P: 0.033585575305464355 Loss: 1288298528.0
ID: 4060018970.0 P: 0.7296554464299441 Loss: 0.0
ID: 4060018990.0 P: 0.17565562060255901 Loss: 0.0
ID: 4060019410.0 P: 0.8631789223498866 Loss: 0.0
ID: 4060019420.0 P: 0.5414612202490917 Loss: 0.0
ID: 4060019710.0 P: 0.2997118905373848 Loss: 0.0
ID: 4060020980.0 P: 0.

array([1.91718206e+10])